# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

print(geoapify_key)

91bbe9337d5241559f0714fe7db40d0c


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,victoria,22.2855,114.1577,21.08,86,95,1.79,HK,1675647602
1,1,dubrovnik,42.6481,18.0922,0.80,29,0,16.98,HR,1675648180
2,2,ipixuna,-1.7625,-48.8043,22.74,97,98,1.22,BR,1675648181
3,3,tasiilaq,65.6145,-37.6368,-3.91,94,100,2.44,GL,1675648096
4,4,kapaa,22.0752,-159.3190,26.18,81,100,4.12,US,1675648181


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600 ,
    size= 'Humidity',
    color= 'City'
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Between 15.5 and 24 degrees celsius
# Between 40% and 60% humidity 
filtered_city_data = city_data_df[(city_data_df['Max Temp'] >= 15.5) & (city_data_df['Max Temp'] <= 24) & (city_data_df['Humidity'] >= 40) & (city_data_df['Humidity'] <= 60)]

# Drop any rows with null values
filtered_city_data.dropna(inplace=True)

# Display sample data
filtered_city_data

c:\Users\Patrick\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,mizan teferi,6.9833,35.5833,19.01,53,62,1.09,ET,1675648183
49,49,hobart,-42.8794,147.3294,22.37,54,75,8.23,AU,1675648057
56,56,mataura,-46.1927,168.8643,19.20,57,98,5.13,NZ,1675647951
60,60,telimele,10.9167,-13.3333,20.58,59,96,1.42,GN,1675648195
100,100,kaitangata,-46.2817,169.8464,19.00,59,69,6.46,NZ,1675648207
128,128,new norfolk,-42.7826,147.0587,22.18,54,49,2.24,AU,1675648215
145,145,isiro,2.7739,27.6160,20.25,43,100,1.59,CD,1675648219
184,184,necochea,-38.5473,-58.7368,23.02,49,100,2.37,AR,1675648232
195,195,ponta do sol,32.6667,-17.1000,16.51,60,7,2.29,PT,1675648235
207,207,kunya,12.2233,8.5325,17.09,59,0,2.06,NG,1675648238


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  filtered_city_data[['City' , 'Country', 'Lat' , 'Lng', 'Humidity' ]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df['Lng']

12      35.5833
49     147.3294
56     168.8643
60     -13.3333
100    169.8464
128    147.0587
145     27.6160
184    -58.7368
195    -17.1000
207      8.5325
208     74.0167
247     90.2201
318    141.9667
353    -98.1128
387     14.7147
407    140.7667
432     46.7219
434     73.3333
447     74.4333
460     97.4000
468    147.3532
478     95.5667
577     73.7333
Name: Lng, dtype: float64

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
categories= "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit": limit,
    "filter": '',
    "bias": '',
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'][index]
    longitude = hotel_df['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url , params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url , params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mizan teferi - nearest hotel: ሆቴል ሳላይሽ / Salayesh Hotel
hobart - nearest hotel: Vibe Hotel Hobart
mataura - nearest hotel: No hotel found
telimele - nearest hotel: No hotel found
kaitangata - nearest hotel: Hotel South Otago
new norfolk - nearest hotel: Woodbridge on the Derwent
isiro - nearest hotel: Hôtel Magbetu
necochea - nearest hotel: Cabañas Río Quequén
ponta do sol - nearest hotel: Estalagem Ponta do Sol
kunya - nearest hotel: No hotel found
kankon - nearest hotel: Dreamcatcher
tura - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
mineral wells - nearest hotel: Baker Hotel
bol - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
riyadh - nearest hotel: White Palace Hotel
pali - nearest hotel: Hotel Pawan International
phaltan - nearest hotel: No hotel found
myitkyina - nearest hotel: New Light Hotel
sandy bay - nearest hotel: Wrest Point Hotel
khonsa - nearest hotel: No hotel found
nagaur - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
12,mizan teferi,ET,6.9833,35.5833,53,ሆቴል ሳላይሽ / Salayesh Hotel
49,hobart,AU,-42.8794,147.3294,54,Vibe Hotel Hobart
56,mataura,NZ,-46.1927,168.8643,57,No hotel found
60,telimele,GN,10.9167,-13.3333,59,No hotel found
100,kaitangata,NZ,-46.2817,169.8464,59,Hotel South Otago
128,new norfolk,AU,-42.7826,147.0587,54,Woodbridge on the Derwent
145,isiro,CD,2.7739,27.6160,43,Hôtel Magbetu
184,necochea,AR,-38.5473,-58.7368,49,Cabañas Río Quequén
195,ponta do sol,PT,32.6667,-17.1000,60,Estalagem Ponta do Sol
207,kunya,NG,12.2233,8.5325,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600 ,
    size= 'Humidity',
    color= 'City',
    hover_cols= ['Hotel Name' , 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)